## Preparation

import the modules and removed previous database (if any)

In [5]:
import sqlite3
import pandas as pd
import os

try:
    os.remove('file_path')
except OSError:
    pass


## Create database 

In [6]:
# Note your path to the database may be different 
conn = sqlite3.connect('file_path/airline2.db')

## Create tables 

First create the tables for the airports, carrier and plane-data dataset 

In [7]:
# Note your path to the data may be different 
airports = pd.read_csv("file_path/airports.csv")
carriers = pd.read_csv("file_path")
planes = pd.read_csv("file_path")

airports.to_sql('airports', con = conn, index = False)
carriers.to_sql('carriers', con = conn, index = False)
planes.to_sql('planes', con = conn, index = False)

5029

Then create the table for ontime, which requires several csv files

In [8]:
c = conn.cursor()

c.execute('''
CREATE TABLE ontime (
  Year int,
  Month int,
  DayofMonth int,
  DayOfWeek int,
  DepTime  int,
  CRSDepTime int,
  ArrTime int,
  CRSArrTime int,
  UniqueCarrier varchar(5),
  FlightNum int,
  TailNum varchar(8),
  ActualElapsedTime int,
  CRSElapsedTime int,
  AirTime int,
  ArrDelay int,
  DepDelay int,
  Origin varchar(3),
  Dest varchar(3),
  Distance int,
  TaxiIn int,
  TaxiOut int,
  Cancelled int,
  CancellationCode varchar(1),
  Diverted varchar(1),
  CarrierDelay int,
  WeatherDelay int,
  NASDelay int,
  SecurityDelay int,
  LateAircraftDelay int
)
''')
conn.commit()

for year in range(2006, 2009):
    ontime = pd.read_csv("/Users/asmacmini/Documents/Programming_for_DS/Harvard Dataverse/Assignment 03/"+str(year)+".csv")
    ontime.to_sql('ontime', con = conn, if_exists = 'append', index = False)

conn.commit()

## Run queries 

Query 1: Find model that has the lowest associated average departure delay

In [9]:
c.execute('''
SELECT model AS model, AVG(ontime.DepDelay) AS avg_delay
FROM planes JOIN ontime USING(tailnum)
WHERE ontime.Cancelled = 0 AND ontime.Diverted = 0 AND ontime.DepDelay > 0
GROUP BY model
ORDER BY avg_delay
''')

# get the first row using the fetchone() method
print(c.fetchone())

#only print out the plane model using index [0]
print(c.fetchone()[0], "has the lowest associated average departure delay.")

('737-230', 12.956403269754768)
737-282 has the lowest associated average departure delay.


Query 2: Find the city that has the highest number of inbound flights (excluding canceled flights)

In [10]:
c.execute('''
SELECT airports.city AS city, COUNT(*) AS total
FROM airports JOIN ontime ON ontime.dest = airports.iata
WHERE ontime.Cancelled = 0
GROUP BY airports.city
ORDER BY total DESC
''')

print(c.fetchone()[0], "has the highest number of inbound flights (excluding canceled flights)")

Chicago has the highest number of inbound flights (excluding canceled flights)


Query 3: Find the carrier that has the highest number of canceled flights

In [11]:
c.execute('''
SELECT carriers.Description AS carrier, COUNT(*) AS total
FROM carriers JOIN ontime ON ontime.UniqueCarrier = carriers.Code
WHERE ontime.Cancelled = 1
AND carriers.Description IN ('United Air Lines Inc.', 'American Airlines Inc.', 'Pinnacle Airlines Inc.', 'Delta Air Lines Inc.')
GROUP BY carriers.Description
ORDER BY total DESC
''')

print(c.fetchone()[0],"has the highest number of canceled flights")

American Airlines Inc. has the highest number of canceled flights


Query 4: Find the carrier that has the highest number of canceled flights, relative to their number of total flights

In [12]:
c.execute('''
SELECT
q1.carrier AS carrier, (CAST(q1.numerator AS FLOAT)/ CAST(q2.denominator AS FLOAT)) AS ratio
FROM
(
  SELECT carriers.Description AS carrier, COUNT(*) AS numerator
  FROM carriers JOIN ontime ON ontime.UniqueCarrier = carriers.Code
  WHERE ontime.Cancelled = 1 AND carriers.Description IN ('United Air Lines Inc.', 'American Airlines Inc.', 'Pinnacle Airlines Inc.', 'Delta Air Lines Inc.')
  GROUP BY carriers.Description
) AS q1 JOIN
(
  SELECT carriers.Description AS carrier, COUNT(*) AS denominator
  FROM carriers JOIN ontime ON ontime.UniqueCarrier = carriers.Code
  WHERE carriers.Description IN ('United Air Lines Inc.', 'American Airlines Inc.', 'Pinnacle Airlines Inc.', 'Delta Air Lines Inc.')
  GROUP BY carriers.Description
) AS q2 USING(carrier)
ORDER BY ratio DESC
''')
print(c.fetchone()[0], "has the highest number of canceled flights, relative to their number of total flights")

Pinnacle Airlines Inc. has the highest number of canceled flights, relative to their number of total flights


Remember to close the connection

In [30]:
conn.close()